1. Load Module

In [33]:
# load modules
train_dir = '../DATA/lang_data/train/'
test_dir = '../DATA/lang_data/test/'

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score

2. Load Data / Preprocess Data into Dataloader

In [48]:
# 1. test_df_list
def dataframe_func(file_dir):
    for root, dirs, files in os.walk(file_dir):
        data_list = []
        label_list = []
        
        for file in files:
            f = open(os.path.join(root, file), 'r')
            data = f.read()
            f.close()
            
            data = data.split()
            data_num = [list(map(ord, list(word.lower()))) for word in data if word.isalpha()]
            data_list.append(data_num)
            
            if 'en' in file:
                label_list.append(0)
            elif 'fr' in file:
                label_list.append(1)
            elif 'id' in file:
                label_list.append(2)
            elif 'tl' in file:
                label_list.append(3)

    # len_col, len_row = max([len(max(data, key = len)) for data in data_list]), max([len(data) for data in data_list])
    # 길이는 우선 지정하자 : col = 45, row = 16100
    len_col, len_row = 45, 16100
    df_list = []
    for idx, data in enumerate(data_list):
        data_frame = pd.DataFrame(0, index = range(len_row), columns = range(len_col))
        
        for idx2, word in enumerate(data):
            for idx3, i in enumerate(word):
                data_frame.loc[idx2, idx3] = i
        
        df_list.append(data_frame.values.reshape(-1))   # 1차원으로 reshape, list에 추가
    
    return df_list, label_list

길이를 우선 최대 길이로 train, test 맞춤

In [52]:
# 3) dataset class 생성 : __len__, __getitem__ 구현
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class LangDataset(Dataset):
    def __init__(self, x, y):
        super(LangDataset, self).__init__()
        self.data = torch.tensor(np.array(x), dtype = torch.float32)
        self.labels = torch.tensor(y, dtype = torch.long)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data = self.data[idx]
        labels = self.labels[idx]
        return data, labels


In [62]:
# 1. data, label 생성
train_data, train_label = dataframe_func(train_dir)
test_data, test_label = dataframe_func(test_dir)

# 2. dataset 생성
train_dataset = LangDataset(train_data, train_label)
test_dataset = LangDataset(test_data, test_label)

# 3. dataloader 생성
Batchs = 8
generator = torch.Generator().manual_seed(11)
train_loader = DataLoader(train_dataset, batch_size = Batchs, shuffle = True, generator = generator)
test_loader = DataLoader(test_dataset, batch_size = Batchs, shuffle = True, generator = generator)

3. Define Model

In [54]:
# 1. 모델 생성
# - basic하게, relu, linear만 사용
# - hidden_layer 변수로 조절
import torch.nn as nn
import torch.nn.functional as F

class LangModel(nn.Module):
    def __init__(self, IN, Hidden_list, OUT):
        super(LangModel, self).__init__()
        self.in_layer = nn.Linear(IN, Hidden_list[0])
        self.hidden = nn.ModuleList()
        for idx, hidden in enumerate(Hidden_list[:-1]):
            self.hidden.append(nn.Linear(hidden, Hidden_list[idx+1]))
        self.out_layer = nn.Linear(Hidden_list[-1], OUT)
        
    def forward(self, x):
        y = F.relu(self.in_layer(x))   # IN -> Hidden_list[0]
        for hidden in self.hidden:
            y = F.relu(hidden(y)) # Hidden_list[0] -> Hidden_list[1] -> ... -> Hidden_list[-1]
        y = self.out_layer(y)           # Hidden_list[-1] -> OUT
        return y

4. Train & Evaluate Model

In [67]:
# 학습 / 평가 종합 함수
# TODO : .train(), .eval() 적용한 함수 구현
# - training 구성 요소 : .train(), for batch

def train_eval(model, train_dl, test_dl, Epochs, lr = 0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr)
    
    for epoch in range(1, Epochs+1):
        # training
        model.train()
        for inputs, labels in train_dl.dataset:
            outputs = model(inputs)
            outputs = outputs.view(-1, 4)   # shape 맞추기
            labels = labels.view(-1)        # shape 맞추기
            # print(outputs, labels, outputs.shape, labels.shape)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        print(f'Epoch : {epoch}, final Loss : {loss}')

        scheduler = ReduceLROnPlateau(optimizer, 'min', factor = 0.1, patience = 5)
        scheduler.step(loss)   # loss가 감소하지 않으면 학습률을 줄임
        # scheduler 정리
            # - optimizer : 최적화할 optimizer
            # - mode : min / max 중 하나 선택, min: 감소, max: 증가
            # - factor : 학습률을 줄일 비율; new_lr = lr * factor
            # - patience : 성능이 증가하지 않는 epoch 수
            # - verbose : True로 설정하면 감소되는 학습률 출력
            
        if scheduler.num_bad_epochs > scheduler.patience:
            print('Early Stopping : Over patience limit')
            break
            
    # evaluation
    model.eval()
    with torch.no_grad():
        mean_acc, mean_f1 = 0, 0
        for idx, (inputs, labels) in enumerate(test_dl.dataset):
            outputs = model(inputs)
            outputs = outputs.view(-1, 4)
            labels = labels.view(-1)
            pred = torch.argmax(outputs, 1)
            acc = accuracy_score(labels, pred)
            f1 = f1_score(labels, pred, average = 'macro')
            print(f'Prediction : {pred}, Label : {labels}')
            print(f'Accuracy : {acc}, F1 : {f1}')
            mean_acc += acc
            mean_f1 += f1
            
    
    print('Finished Training')
    print(f'Epoch : {epoch}, Loss : {loss:.4f}, Total Accuracy : {mean_acc/len(test_dl.dataset)}, Total F1 : {mean_f1/len(test_dl.dataset)}')

In [68]:
len(test_loader.dataset)

16

In [69]:
train_data[0].shape, test_data[0].shape, len(train_data[0])
# 아 이거 shape 맞춰야 하네
# 일단 돌려본다

((724500,), (724500,), 724500)

In [70]:
IN = len(train_data[0])
model = LangModel(IN, [64, 32, 16], 4)
train_eval(model, train_loader, test_loader, 50, 0.001)

Epoch : 1, final Loss : 1.4765543937683105
Epoch : 2, final Loss : 1.4773564338684082
Epoch : 3, final Loss : 1.476163625717163
Epoch : 4, final Loss : 1.474111795425415
Epoch : 5, final Loss : 1.4720945358276367
Epoch : 6, final Loss : 1.4700841903686523
Epoch : 7, final Loss : 1.4680943489074707
Epoch : 8, final Loss : 1.466133713722229
Epoch : 9, final Loss : 1.464207410812378
Epoch : 10, final Loss : 1.462318778038025
Epoch : 11, final Loss : 1.4604696035385132
Epoch : 12, final Loss : 1.4586613178253174
Epoch : 13, final Loss : 1.4568941593170166
Epoch : 14, final Loss : 1.4551684856414795
Epoch : 15, final Loss : 1.4534841775894165
Epoch : 16, final Loss : 1.4518406391143799
Epoch : 17, final Loss : 1.45023775100708
Epoch : 18, final Loss : 1.4486744403839111
Epoch : 19, final Loss : 1.447150468826294
Epoch : 20, final Loss : 1.445664644241333
Epoch : 21, final Loss : 1.4442164897918701
Epoch : 22, final Loss : 1.4428050518035889
Epoch : 23, final Loss : 1.4414297342300415
Epoch 

새로운 데이터로 예측해보기

In [84]:
# 프랑스어로 장문의 아무말 입력
new_data = 'Je suis un homme de 30 ans. Français, je suis né à Paris. J\'aime la musique et le sport. Je suis célibataire. Je suis un homme de 30 ans. Français, je suis né à Paris. J\'aime la musique et le sport. Je suis célibataire. Je suis un homme de 30 ans. Français, je suis né à Paris. J\'aime la musique et le sport. Je suis célibataire. Je suis un homme de 30 ans. Français, je suis né à Paris. J\'aime la musique et le sport. Je suis célibataire.'

id_data = 'Saya seorang pria berusia 30 tahun. Saya lahir di Jakarta. Saya suka musik dan olahraga. Saya lajang. Saya seorang pria berusia 30 tahun. Saya lahir di Jakarta. Saya suka musik dan olahraga. Saya lajang. Saya seorang pria berusia 30 tahun. Saya lahir di Jakarta. Saya suka musik dan olahraga. Saya lajang. Saya seorang pria berusia 30 tahun. Saya lahir di Jakarta. Saya suka musik dan olahraga. Saya lajang.'

id_data2 = 'Bahasa Indonesia ([baˈhasa indoˈnesija]) merupakan bahasa resmi sekaligus bahasa nasional di Indonesia.[15] Bahasa Indonesia merupakan varietas yang dibakukan dari bahasa Melayu,[16] sebuah bahasa rumpun Austronesia yang digolongkan kedalam rumpun Melayik yang sendirinya merupakan cabang turunan dari cabang Melayu-Polinesia. Bahasa Indonesia telah sejak lama digunakan sebagai basantara di wilayah kepulauan Indonesia yang rata-rata memiliki kemajemukan linguistika. Dengan jumlah penutur bahasa yang lumayan besar ditambah dengan populasi diaspora yang tinggal di luar negeri, bahasa Indonesia masuk sebagai salah satu bahasa yang paling banyak digunakan atau dituturkan di seluruh duniosakata bahasa Indonesia juga dipengaruhi oleh beberapa bahasa lokal di wilayah kepulauan Indonesia (misalnya: bahasa Jawa, Minangkabau, Bugis, Makasar, dan lain sebagainya),[18] serta dari bahasa asing yang disebabkan oleh kontak sejarah dan keterikatan sejarah dengan bahasa lain dari wilayah lainnya.[19] Bahasa Indonesia memiliki banyak kata serapan yang berasal dari bahasa-bahasa Eropa, terutama dari bahasa Belanda, Portugis, Spanyol, dan Inggris. Bahasa Indonesia juga memiliki kata serapan yang berasal dari bahasa Sanskerta, Tionghoa, dan Arab yang membaur menjadi elemen dalam bahasa Indonesia yang terpengaruh karena adanya faktor-faktor seperti aktivitas perdagangan maupun religius yang telah berlangsung sejak zaman kuno di wilayah kepulauan Indonesia.'

In [82]:
def frame_func(data):
    len_col, len_row = 45, 16100
    data_frame = pd.DataFrame(0, index = range(len_row), columns = range(len_col))
    
    data = data.split()
    data_num = [list(map(ord, list(word.lower()))) for word in data if word.isalpha()]
    
    for idx2, word in enumerate(data_num):
        for idx3, i in enumerate(word):
            data_frame.loc[idx2, idx3] = i
        
    data_frame = data_frame.values.reshape(-1)
    data_frame = torch.tensor(data_frame, dtype = torch.float32)
    
    return data_frame

In [86]:
# prediction
# - {0: 'en', 1: 'fr', 2: 'id', 3: 'tl'}
def predict(data):
    tensor_data = frame_func(data)
    
    model.eval()
    pred = model(tensor_data)
    pred = torch.argmax(pred)
    if pred == 0:
        print('영어')
    elif pred == 1:
        print('프랑스어')
    elif pred == 2:
        print('인도네시아어')
    elif pred == 3:
        print('필리핀어')
    else:
        print('Error')

In [87]:
predict(new_data), predict(id_data), predict(id_data2)

영어
영어
영어


(None, None, None)

ㅠㅠ 다 영어로 나오는 듯